In [71]:
import numpy as np
from collections import Counter, deque
from scipy.ndimage import label

## Part 1

In [2]:
def read_text_file_as_grid(file_path):
    with open(file_path, 'r') as file:
        grid = [list(line.strip()) for line in file if line.strip()]
    return np.array(grid)

In [3]:
input_file = "example1.txt"
grid = read_text_file_as_grid(input_file)

In [ ]:
def calculate_edges(region, grid):
    edges = 0
    for x, y in zip(*np.where(region)):
        if x == 0 or grid[x-1, y] != grid[x, y]:  
            edges += 1
        if x == grid.shape[0]-1 or grid[x+1, y] != grid[x, y]: 
            edges += 1
        if y == 0 or grid[x, y-1] != grid[x, y]:
            edges += 1
        if y == grid.shape[1]-1 or grid[x, y+1] != grid[x, y]:
            edges += 1
    return edges

def find_regions_and_edges(grid):
    """
    Identifies regions in the grid and computes their sizes and edge counts.
    """
    unique_chars = np.unique(grid)
    results = {}

    for char in unique_chars:
        mask = (grid == char)

        labeled_array, num_features = label(mask)
        
        for region_id in range(1, num_features + 1):
            region = (labeled_array == region_id)
            
            num_elements = np.sum(region)
            num_edges = calculate_edges(region, grid)
            
            results[f'{char}-{region_id}'] = {
                'elements': num_elements,
                'edges': num_edges
            }
    
    return results

results = find_regions_and_edges(grid)
for region, stats in results.items():
    print(f"Region {region}: {stats}")


Region C-1: {'elements': np.int64(14), 'edges': 28}
Region C-2: {'elements': np.int64(1), 'edges': 4}
Region E-1: {'elements': np.int64(13), 'edges': 18}
Region F-1: {'elements': np.int64(10), 'edges': 18}
Region I-1: {'elements': np.int64(4), 'edges': 8}
Region I-2: {'elements': np.int64(14), 'edges': 22}
Region J-1: {'elements': np.int64(11), 'edges': 20}
Region M-1: {'elements': np.int64(5), 'edges': 12}
Region R-1: {'elements': np.int64(12), 'edges': 18}
Region S-1: {'elements': np.int64(3), 'edges': 8}
Region V-1: {'elements': np.int64(13), 'edges': 20}


In [ ]:
def find_regions_and_edges_manual(grid):
    rows, cols = grid.shape
    visited = np.zeros_like(grid, dtype=bool)
    results = {}
    region_id = 1
    
    def calculate_cell_edges(x, y, char):
        edges = 0
        for nx, ny in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
            if 0 <= nx < rows and 0 <= ny < cols:
                if grid[nx, ny] != char: 
                    edges += 1
            else:
                edges += 1
        return edges
    
    def bfs(x, y, char):
        queue = deque([(x, y)])
        visited[x, y] = True
        size = 0
        total_edges = 0
        
        while queue:
            cx, cy = queue.popleft()
            size += 1
            total_edges += calculate_cell_edges(cx, cy, char)
            
            # Check 4-connected neighbors
            for nx, ny in [(cx-1, cy), (cx+1, cy), (cx, cy-1), (cx, cy+1)]:
                if 0 <= nx < rows and 0 <= ny < cols:  # In bounds
                    if not visited[nx, ny] and grid[nx, ny] == char:
                        visited[nx, ny] = True
                        queue.append((nx, ny))
        
        return size, total_edges
    
    for i in range(rows):
        for j in range(cols):
            if not visited[i, j]:  # Found a new region
                char = grid[i, j]
                size, edges = bfs(i, j, char)
                results[f'{char}-{region_id}'] = {'elements': size, 'edges': edges}
                region_id += 1
    
    return results

# results = find_regions_and_edges_manual(grid)
# for region, stats in results.items():
#     print(f"Region {region}: {stats}")

In [9]:
input_file = "input.txt"
grid = read_text_file_as_grid(input_file)

results = find_regions_and_edges_manual(grid)

total_cost = 0
for stats in results.values():
    price = stats["elements"] * stats["edges"]
    total_cost += price
    
print(total_cost)

1421958


## Part 2

In [72]:
def find_region_edges(x, y, grid, visited):
    """
    Perform BFS to find all cells in the region starting from (x, y).
    Marks visited cells and returns the region size.
    """
    char = grid[x, y]
    region = np.zeros_like(grid, dtype=bool)
    queue = deque([(x, y)])
    visited[x, y] = True 
    size = 0 
    
    while queue:
        cx, cy = queue.popleft()
        region[cx, cy] = True
        size += 1
        
        for nx, ny in [(cx-1, cy), (cx+1, cy), (cx, cy-1), (cx, cy+1)]:
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1]:
                if not visited[nx, ny] and grid[nx, ny] == char:
                    visited[nx, ny] = True
                    queue.append((nx, ny))
    
    return region, size


In [64]:
def calculate_corners(region, grid):
    """
    Calculates the number of corners for a region in the grid.
    A corner is a location where a boundary of the region changes direction.
    """
    corners = 0
    rows, cols = grid.shape
    
    # Iterate over all cells in the region
    for x, y in zip(*np.where(region)):
        char = grid[x, y]
        
        # Check the 8 surrounding cells to determine if there's a boundary change
        neighbors = [
            (x-1, y), (x+1, y), (x, y-1), (x, y+1),  # up, down, left, right
            (x-1, y-1), (x-1, y+1), (x+1, y-1), (x+1, y+1)  # 4 diagonals
        ]
        
        print(f"char {char} at {x,y}")
        
        # To determine if it's a corner, we check if there is a change in boundary direction
        boundary_changes = 0
        for nx, ny in neighbors:
            if 0 <= nx < rows and 0 <= ny < cols:
                if grid[nx, ny] != char:  # Neighbor is outside the region
                    print(f"boundary change at: {nx, ny}, {grid[nx, ny]}")
                    boundary_changes += 1
        
        # A corner requires at least two changes in direction
        if boundary_changes >= 2:
            corners += 1

    return corners

In [77]:
def count_corners(grid, region):
    rows, cols = grid.shape
    corner_count = 0
    
    # Iterate over all cells in the region
    for x, y in zip(*np.where(region)):
        char = grid[x, y]
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)] 

        count_same = 0
        count_diff = 0
        count_end = 0
        corner = 0

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            
            if 0 <= nx < rows and 0 <= ny < cols:
                if char == grid[nx, ny]:
                    count_same += 1
                else: 
                    count_diff += 1
            else:
                count_end += 1
        
        # Logic for corner detection based on surrounding cells
        if count_same == 0:
            corner = 4
        elif count_same == 1 and (count_diff == 2 or count_end == 2):
            corner = 2
        elif count_same == 2 and count_end == 2:
            corner = 2
        elif count_same == 3:
            corner = 2
        else:
            corner
            
    corner_count += corner
    
    return corner_count

In [78]:
input_file = "example2.txt"
grid = read_text_file_as_grid(input_file)
visited = np.zeros_like(grid, dtype=bool)

region_info = []

for x in range(grid.shape[0]):
    for y in range(grid.shape[1]):
        if not visited[x, y]:
            # Get region and its size
            region, size = find_region_edges(x, y, grid, visited)
            corners = count_corners(region, grid)
            region_info.append((size, corners))

# total_cost = 0
# for size, corner in region_info:
#     price = size * corner
#     total_cost += price
    
print(region_info)

[(17, 2), (4, 2), (4, 2)]
